In [29]:
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# Loading MNIST Data
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Reshaping to format which CNN expects (batch, height, width, channels)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1).astype('float32')

# normalize inputs from 0-255 to 0-1
X_train/=255
X_test/=255

# one hot encode
number_of_classes = 10
y_train = np_utils.to_categorical(y_train, number_of_classes)
y_test = np_utils.to_categorical(y_test, number_of_classes)

# Model Creation
model = Sequential()
model.add(Conv2D(32, (5, 5), input_shape=(X_train.shape[1], X_train.shape[2], 1), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(number_of_classes, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)

# Save the model
model.save('mnistCNN.h5')

# Final evaluation of the model
metrics = model.evaluate(X_test, y_test, verbose=0)
print("Metrics(Test loss & Test Accuracy): ")
print(metrics)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 36s 594us/step - loss: 0.2872 - accuracy: 0.9172 - val_loss: 0.0712 - val_accuracy: 0.9779
Epoch 2/10
60000/60000 [==============================] - 35s 583us/step - loss: 0.0767 - accuracy: 0.9768 - val_loss: 0.0467 - val_accuracy: 0.9846
Epoch 3/10
60000/60000 [==============================] - 34s 571us/step - loss: 0.0574 - accuracy: 0.9827 - val_loss: 0.0354 - val_accuracy: 0.9886
Epoch 4/10
60000/60000 [==============================] - 35s 585us/step - loss: 0.0467 - accuracy: 0.9853 - val_loss: 0.0313 - val_accuracy: 0.9888
Epoch 5/10
60000/60000 [==============================] - 34s 569us/step - loss: 0.0393 - accuracy: 0.9876 - val_loss: 0.0351 - val_accuracy: 0.9885
Epoch 6/10
60000/60000 [==============================] - 34s 561us/step - loss: 0.0331 - accuracy: 0.9898 - val_loss: 0.0235 - val_accuracy: 0.9920
Epoch 7/10
60000/60000 [==============================] 

In [45]:
from tkinter import *
from tkinter.ttk import *
import PIL.ImageGrab
from PIL import Image
from keras.models import load_model
import numpy as np

# load model
model = load_model('mnistCNN.h5')

# create global variables 
operator = "Jose John Predicts: "
operator2 = ""

# create function to clear canvas and text
def Clear():
    cv.delete("all")
    global operator2
    text_input.set(operator2)

# create function to predict and display predicted number
def Predict():
    file = 'C:/Users/vellaj/Desktop/Python/image.jpg'
    if file:
        # save the canvas in jpg format
        x = root.winfo_rootx() + cv.winfo_x()
        y = root.winfo_rooty() + cv.winfo_y()
        x1 = x + cv.winfo_width()
        y1 = y + cv.winfo_height()
        PIL.ImageGrab.grab().crop((x,y,x1,y1)).save(file)
        
        #convert to greyscale
        img = Image.open(file).convert("L")
        
        #resize image
        img = img.resize((28,28))
        
        #convert image to array
        im2arr = np.array(img)
        
        # reshape array
        im2arr = im2arr.reshape(1,28,28,1)
        
        #predict class
        y_pred2 = model.predict_classes(im2arr)
        
        #covert class to scalar
        x = y_pred2[0]
        
        #display predicted number
        global operator
        operator = operator+str(x)
        text_input.set(operator)
        operator = operator = "Jose John Predicts: "

# create function to draw on canvas
def paint(event):
    old_x = event.x
    old_y = event.y        
        
    cv.create_line(old_x, old_y, event.x, event.y,
                               width=20, fill="white",
                               capstyle=ROUND, smooth=TRUE, splinesteps=36)

# all interface elements must be between Tk() and mainloop()
root = Tk()

#create string variable
text_input = StringVar()

#create field to display text
textdisplay = Entry(root, 
               textvariable = text_input,  
               justify = 'center')

# create predict and clear buttons
btn1 = Button(root, text = "Predict", command = lambda:Predict())
btn2 = Button(root, text = "Clear", command = lambda:Clear())

#create canvas to draw on
cv = Canvas(root,width=200,height=200,bg="black",)

#using left mouse button to draw
cv.bind('<B1-Motion>', paint)

#organise the elements
cv.grid(row = 0, column = 0)
textdisplay.grid(row = 0, column = 1)
btn1.grid(row = 1, column = 0)
btn2.grid(row = 1, column = 1)

#this 2 lines for expand the interface
root.rowconfigure(0, weight=2)
root.columnconfigure(1, weight=2)

root.mainloop()